# 뉴스 요약하고 문서화하기

- pip install --upgrade openai

In [1]:
import os
from dotenv import load_dotenv

# .env 파일 경로 지정 
load_dotenv(override=True)

# Open AI API 키 설정하기
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [2]:
summary_prompt = """당신은 언어 이해 및 요약에 훈련된 고도로 숙련된 AI입니다.
다음 텍스트를 읽고 간결한 추상적 단락으로 요약했으면 합니다.
전체 텍스트를 읽을 필요 없이 토론의 요점을 이해하는 데 도움이 될 수 있는 일관되고
읽을 수 있는 요약을 제공하여 가장 중요한 요점을 유지하는 것을 목표로 합니다.
불필요한 세부 사항이나 접선 사항은 피하십시오."""
key_points_prompt = """당신은 정보를 핵심 포인트로 전달하는 데 특화된 능숙한 AI입니다.
다음 텍스트를 기반으로 논의되거나 언급된 주요 포인트를 확인하고 나열합니다.
이는 논의의 본질에 가장 중요한 아이디어, 결과 또는 주제가 되어야 합니다.
당신의 목표는 누군가가 읽을 수 있는 목록을 제공하여 이야기된 내용을 빠르게 이해하는 것입니다."""
action_items_prompt = """당신은 대화를 분석하고 행동 항목을 추출하는 데 있어 AI 전문가입니다.
본문을 검토하고 합의되거나 수행이 필요하다고 언급된 모든 작업, 과제 또는 행동을 식별하십시오.
이것들은 특정 개인에게 할당된 작업일 수도 있고 그룹이 취하기로 결정한 일반적인 행동일 수도 있습니다.
이러한 행동 항목을 명확하고 간결하게 나열하십시오."""
sentiment_prompt = """당신은 언어와 감정 분석에 전문성을 갖춘 AI로서 당신의 과제는 다음 텍스트의 감
정을 분석하는 것입니다.
토론의 전체적인 톤, 사용된 언어가 전달하는 감정, 단어와 구가 사용되는 맥락을 고려하십시오.
감정이 일반적으로 긍정적인지 부정적인지 중립적인지를 표시하고 가능한 한 당신의 분석에 대해 간략한 설명을 제
공하십시오."""

In [3]:
# OpenAI 객체 생성
from openai import OpenAI
client = OpenAI(api_key = OPENAI_API_KEY)

# 번역 함수

In [4]:
def text_extraction(text, prompt):
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        temperature=0,
        messages=[
        { "role": "system", "content": prompt },
        { "role": "user", "content": text }
        ]
    )
    return response.choices[0].message.content

## 음성 파일 -> text로 변환

In [5]:
audio_file_path = "datas/news.mp3"
with open(audio_file_path, 'rb') as audio_file:
    transcription = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    response_format="text"
)

## summary 하기

In [6]:
abstract_summary = text_extraction(transcription, summary_prompt)
key_points = text_extraction(transcription, key_points_prompt)
action_items = text_extraction(transcription, action_items_prompt)
sentiment = text_extraction(transcription, sentiment_prompt)
news_data = {
    'abstract_summary': abstract_summary,
    'key_points': key_points,
    'action_items': action_items,
    'sentiment': sentiment
}
print(news_data)

{'abstract_summary': '짧은 장마와 긴 폭염으로 인해 말벌 활동이 증가하면서 소방대원들의 벌집 제거 출동이 급증했습니다. 소방대원들은 보호복을 착용하고 스프레이를 사용하여 벌집을 제거하는 작업을 수행하고 있으며, 이로 인해 말벌 쏘임 사고도 빈번하게 발생하고 있습니다. 소방당국은 말벌이 자주 나타나는 지역에서는 주변을 잘 살펴 벌집을 신고하고, 벌에게 공격당할 경우 신속히 현장을 벗어날 것을 권장하고 있습니다. 지난달 벌집 제거 출동은 전년 대비 57% 증가한 44,000건에 달했습니다.', 'key_points': '1. 짧은 장마와 긴 폭염으로 인해 말벌 활동이 증가하였습니다.\n2. 소방대원들이 말벌 제거 작업에 더 자주 출동하고 있으며, 일일 최대 15건까지 출동하는 경우도 있습니다.\n3. 말벌 제거 출동 건수는 지난해 대비 57% 증가하여 지난달에만 44,000건에 달했습니다.\n4. 소방대원들은 말벌 제거 시 특수 보호복을 착용하고, 벌집을 스프레이로 처리합니다.\n5. 소방당국은 말벌이 자주 나타나는 지역에서는 주변을 잘 살펴 벌집을 신고하고, 벌 공격 시 빠르게 현장을 벗어날 것을 조언합니다.', 'action_items': '1. 소방대원들은 말벌집 제거 작업을 계속 수행합니다.\n2. 시민들은 말벌이 자주 나타나는 지역에서 벌집을 발견하면 즉시 소방당국에 신고해야 합니다.\n3. 말벌 공격을 받을 경우, 시민들은 현장에서 약 20미터 정도 빠르게 벗어나야 합니다.', 'sentiment': '이 텍스트의 전체적인 감정은 주로 중립적이며 경고적인 요소가 포함되어 있습니다. 텍스트는 말벌의 증가와 이로 인한 소방대원의 활동 증가를 설명하고 있으며, 이는 자연 현상과 그에 따른 사회적 대응을 보고하는 뉴스 보도의 특성을 반영합니다. 글에서는 말벌로 인한 위험과 소방대원들의 대응 과정을 상세히 설명하고 있어, 독자에게 정보를 제공하고 주의를 촉구하는 목적이 강합니다.\n\n감정적으로는 긴장감이 느껴지기도 하지만, 전반적으로는 사실적

In [7]:
import json
print(json.dumps(news_data, ensure_ascii=False, indent=4))

{
    "abstract_summary": "짧은 장마와 긴 폭염으로 인해 말벌 활동이 증가하면서 소방대원들의 벌집 제거 출동이 급증했습니다. 소방대원들은 보호복을 착용하고 스프레이를 사용하여 벌집을 제거하는 작업을 수행하고 있으며, 이로 인해 말벌 쏘임 사고도 빈번하게 발생하고 있습니다. 소방당국은 말벌이 자주 나타나는 지역에서는 주변을 잘 살펴 벌집을 신고하고, 벌에게 공격당할 경우 신속히 현장을 벗어날 것을 권장하고 있습니다. 지난달 벌집 제거 출동은 전년 대비 57% 증가한 44,000건에 달했습니다.",
    "key_points": "1. 짧은 장마와 긴 폭염으로 인해 말벌 활동이 증가하였습니다.\n2. 소방대원들이 말벌 제거 작업에 더 자주 출동하고 있으며, 일일 최대 15건까지 출동하는 경우도 있습니다.\n3. 말벌 제거 출동 건수는 지난해 대비 57% 증가하여 지난달에만 44,000건에 달했습니다.\n4. 소방대원들은 말벌 제거 시 특수 보호복을 착용하고, 벌집을 스프레이로 처리합니다.\n5. 소방당국은 말벌이 자주 나타나는 지역에서는 주변을 잘 살펴 벌집을 신고하고, 벌 공격 시 빠르게 현장을 벗어날 것을 조언합니다.",
    "action_items": "1. 소방대원들은 말벌집 제거 작업을 계속 수행합니다.\n2. 시민들은 말벌이 자주 나타나는 지역에서 벌집을 발견하면 즉시 소방당국에 신고해야 합니다.\n3. 말벌 공격을 받을 경우, 시민들은 현장에서 약 20미터 정도 빠르게 벗어나야 합니다.",
    "sentiment": "이 텍스트의 전체적인 감정은 주로 중립적이며 경고적인 요소가 포함되어 있습니다. 텍스트는 말벌의 증가와 이로 인한 소방대원의 활동 증가를 설명하고 있으며, 이는 자연 현상과 그에 따른 사회적 대응을 보고하는 뉴스 보도의 특성을 반영합니다. 글에서는 말벌로 인한 위험과 소방대원들의 대응 과정을 상세히 설명하고 있어, 독자에게 정보를 제공하고 주의를 촉구하는 목적이 강합니다.\n\n감정적으로는 긴장감이 느껴지기

In [8]:
for innews_data in news_data:
    print(f"{innews_data}:\n{news_data[innews_data]}\n")

abstract_summary:
짧은 장마와 긴 폭염으로 인해 말벌 활동이 증가하면서 소방대원들의 벌집 제거 출동이 급증했습니다. 소방대원들은 보호복을 착용하고 스프레이를 사용하여 벌집을 제거하는 작업을 수행하고 있으며, 이로 인해 말벌 쏘임 사고도 빈번하게 발생하고 있습니다. 소방당국은 말벌이 자주 나타나는 지역에서는 주변을 잘 살펴 벌집을 신고하고, 벌에게 공격당할 경우 신속히 현장을 벗어날 것을 권장하고 있습니다. 지난달 벌집 제거 출동은 전년 대비 57% 증가한 44,000건에 달했습니다.

key_points:
1. 짧은 장마와 긴 폭염으로 인해 말벌 활동이 증가하였습니다.
2. 소방대원들이 말벌 제거 작업에 더 자주 출동하고 있으며, 일일 최대 15건까지 출동하는 경우도 있습니다.
3. 말벌 제거 출동 건수는 지난해 대비 57% 증가하여 지난달에만 44,000건에 달했습니다.
4. 소방대원들은 말벌 제거 시 특수 보호복을 착용하고, 벌집을 스프레이로 처리합니다.
5. 소방당국은 말벌이 자주 나타나는 지역에서는 주변을 잘 살펴 벌집을 신고하고, 벌 공격 시 빠르게 현장을 벗어날 것을 조언합니다.

action_items:
1. 소방대원들은 말벌집 제거 작업을 계속 수행합니다.
2. 시민들은 말벌이 자주 나타나는 지역에서 벌집을 발견하면 즉시 소방당국에 신고해야 합니다.
3. 말벌 공격을 받을 경우, 시민들은 현장에서 약 20미터 정도 빠르게 벗어나야 합니다.

sentiment:
이 텍스트의 전체적인 감정은 주로 중립적이며 경고적인 요소가 포함되어 있습니다. 텍스트는 말벌의 증가와 이로 인한 소방대원의 활동 증가를 설명하고 있으며, 이는 자연 현상과 그에 따른 사회적 대응을 보고하는 뉴스 보도의 특성을 반영합니다. 글에서는 말벌로 인한 위험과 소방대원들의 대응 과정을 상세히 설명하고 있어, 독자에게 정보를 제공하고 주의를 촉구하는 목적이 강합니다.

감정적으로는 긴장감이 느껴지기도 하지만, 전반적으로는 사실적인 보도의 특성상 감정 표현이 크게 드러나지

## doc 파일로 저장하기

- pip install python-docx

In [11]:
from docx import Document
doc = Document()

for key, value in news_data.items():
    # _로 나눈 단어들을 대문자로 만들고 공백으로 연결
    heading = ' '.join(word.capitalize() for word in key.split('_'))
    doc.add_heading(heading, level=1)
    doc.add_paragraph(value)
    # 섹션 사이에 줄바꿈 추가
    doc.add_paragraph()
    
# 워드 파일로 데이터 저장
import os

folder_path = "output"
os.makedirs(folder_path, exist_ok=True)
doc.save('output/news_minutes.docx')